In [1]:
from models.PCA import PCA
from models.FF import FF

import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import charas
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
def model_inference_and_predict(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = []
    predict_result = []
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            inference_result.append(model.inference(m))
            predict_result.append(model.predict(m))
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')

In [3]:
pca_1 = PCA(K=1, portfolio=True)
pca_2 = PCA(K=2, portfolio=True)
pca_3 = PCA(K=3, portfolio=True)
pca_4 = PCA(K=4, portfolio=True)
pca_5 = PCA(K=5, portfolio=True)
pca_6 = PCA(K=6, portfolio=True)

In [4]:
model_inference_and_predict(pca_1)
# model_inference_and_predict(pca_2)
# model_inference_and_predict(pca_3)
# model_inference_and_predict(pca_4)
# model_inference_and_predict(pca_5)
# model_inference_and_predict(pca_6)

PCA_1 Inferencing & Predicting: 100%|██████████| 30/30 [00:25<00:00,  1.17it/s, Year=2016]


In [5]:
ff_1 = FF(K=1, portfolio=True)
ff_2 = FF(K=2, portfolio=True)
ff_3 = FF(K=3, portfolio=True)
ff_4 = FF(K=4, portfolio=True)
ff_5 = FF(K=5, portfolio=True)
ff_6 = FF(K=6, portfolio=True)

In [6]:
model_inference_and_predict(ff_1)
model_inference_and_predict(ff_2)
model_inference_and_predict(ff_3)
model_inference_and_predict(ff_4)
model_inference_and_predict(ff_5)
model_inference_and_predict(ff_6)

FF_6 Inferencing & Predicting: 100%|██████████| 30/30 [00:03<00:00,  7.97it/s, Year=2016]


In [7]:
def model_inference_and_predict_CA(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = pd.DataFrame()
    predict_result = pd.DataFrame()
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    stock_index = pd.Series(dtype=np.float64)
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            m_stock_index, _, _, _ = model._get_item(m)
            print(m_stock_index)
            stock_index = pd.concat([stock_index, pd.Series(m_stock_index)]).drop_duplicates()
            inference_R = model.inference(m) # return (N, 1)
            predict_R = model.inference(m) # reutrn (N, 1)
            # move from cuda to cpu
            inference_R = inference_R.cpu().detach().numpy()
            predict_R = predict_R.cpu().detach().numpy()
            inference_R = pd.DataFrame(inference_R, index=m_stock_index, columns=[g[0]]).reindex(stock_index)
            predict_R = pd.DataFrame(predict_R, index=m_stock_index, columns=[g[0]]).reindex(stock_index)
            
            inference_result = pd.concat([inference_result, inference_R], axis=1) # (N, T)
            predict_result = pd.concat([predict_result, predict_R], axis=1) # (N, T)
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference_stock.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict_stock.csv')

In [8]:
from models.CA import CA0, CA1, CA2, CA3

In [9]:
ca3_1 = CA3(1).to('cuda')
# ca3_2 = CA3(2).to('cuda')
# ca3_3 = CA3(3).to('cuda')
# ca3_4 = CA3(4).to('cuda')
# ca3_5 = CA3(5).to('cuda')
# ca3_6 = CA3(6).to('cuda')

In [10]:
model_inference_and_predict_CA(ca3_1)
# model_inference_and_predict_CA(ca3_2)
# model_inference_and_predict_CA(ca3_3)
# model_inference_and_predict_CA(ca3_4)
# model_inference_and_predict_CA(ca3_5)
# model_inference_and_predict_CA(ca3_6)


CA3 Inferencing & Predicting:   0%|          | 0/30 [00:00<?, ?it/s, Year=1987]

Test loss: 283.30841064453125
Predicted: tensor([-0.7381, -0.7114, -0.8125,  ..., -0.7915, -0.7824, -0.7771],
       device='cuda:0', grad_fn=<SumBackward1>)
Ground truth: tensor([ 1.6677,  3.2437, -5.3896,  ..., 23.9302, -3.2769,  4.8032],
       device='cuda:0')
Index([10000.0, 10001.0, 10002.0, 10003.0, 10005.0, 10007.0, 10008.0, 10009.0,
       10010.0, 10012.0,
       ...
       93121.0, 93156.0, 93164.0, 93201.0, 93220.0, 93236.0, 93252.0, 93279.0,
       93287.0, 93316.0],
      dtype='float64', name='permno', length=6600)
Index([10000.0, 10001.0, 10002.0, 10003.0, 10005.0, 10007.0, 10008.0, 10009.0,
       10010.0, 10012.0,
       ...
       93121.0, 93156.0, 93164.0, 93201.0, 93220.0, 93236.0, 93252.0, 93279.0,
       93287.0, 93316.0],
      dtype='float64', name='permno', length=6655)
Index([10000.0, 10001.0, 10002.0, 10003.0, 10005.0, 10007.0, 10008.0, 10009.0,
       10010.0, 10012.0,
       ...
       93121.0, 93156.0, 93164.0, 93201.0, 93220.0, 93236.0, 93252.0, 93279.0,

CA3 Inferencing & Predicting:   3%|▎         | 1/30 [00:34<16:45, 34.67s/it, Year=1988]

Index([10001.0, 10002.0, 10003.0, 10005.0, 10008.0, 10009.0, 10010.0, 10012.0,
       10016.0, 10017.0,
       ...
       93105.0, 93113.0, 93121.0, 93156.0, 93164.0, 93201.0, 93220.0, 93236.0,
       93252.0, 93316.0],
      dtype='float64', name='permno', length=7119)
Early stop at epoch 16
Test loss: 208.09400939941406
Predicted: tensor([1.0000, 1.0249, 1.5692,  ..., 2.4403, 1.2405, 1.8459], device='cuda:0',
       grad_fn=<SumBackward1>)
Ground truth: tensor([ 2.6012, -5.3380, -9.4344,  ..., 20.9686, -9.2835,  2.9883],
       device='cuda:0')
Index([10001.0, 10002.0, 10003.0, 10005.0, 10008.0, 10009.0, 10010.0, 10012.0,
       10016.0, 10017.0,
       ...
       93105.0, 93113.0, 93121.0, 93156.0, 93164.0, 93201.0, 93220.0, 93236.0,
       93252.0, 93316.0],
      dtype='float64', name='permno', length=7108)
Index([10001.0, 10002.0, 10003.0, 10005.0, 10008.0, 10009.0, 10010.0, 10012.0,
       10016.0, 10017.0,
       ...
       93105.0, 93113.0, 93121.0, 93156.0, 93164.0, 93201.0, 

CA3 Inferencing & Predicting:   7%|▋         | 2/30 [01:07<15:33, 33.36s/it, Year=1989]

...

Test loss: 145.5519256591797
Predicted: tensor([0.5716, 0.5852, 0.5197,  ..., 1.0077, 0.6555, 0.6741], device='cuda:0',
       grad_fn=<SumBackward1>)
Ground truth: tensor([-0.0495,  8.6459,  5.8010,  ..., 40.0638,  8.1967, -7.1653],
       device='cuda:0')
Index([10001.0, 10025.0, 10026.0, 10028.0, 10032.0, 10044.0, 10051.0, 10065.0,
       10104.0, 10107.0,
       ...
       93420.0, 93422.0, 93423.0, 93426.0, 93427.0, 93428.0, 93429.0, 93433.0,
       93434.0, 93436.0],
      dtype='float64', name='permno', length=5742)
Index([10001.0, 10025.0, 10026.0, 10028.0, 10032.0, 10044.0, 10051.0, 10065.0,
       10104.0, 10107.0,
       ...
       93420.0, 93422.0, 93423.0, 93426.0, 93427.0, 93428.0, 93429.0, 93433.0,
       93434.0, 93436.0],
      dtype='float64', name='permno', length=5732)
Index([10001.0, 10025.0, 10026.0, 10028.0, 10032.0, 10044.0, 10051.0, 10065.0,
       10104.0, 10107.0,
       ...
       93420.0, 93422.0, 93423.0, 93426.0, 93427.0, 93428.0, 93429.0, 93433.0,
      

CA3 Inferencing & Predicting:  97%|█████████▋| 29/30 [24:29<01:08, 68.74s/it, Year=2016]

Test loss: 273.3381652832031
Predicted: tensor([-2.8138, -3.0033, -1.8221,  ..., -3.5626, -2.3191, -2.0537],
       device='cuda:0', grad_fn=<SumBackward1>)
Ground truth: tensor([-2.2018, -2.6386,  4.3112,  ..., -5.3772, -0.8815, -7.2911],
       device='cuda:0')
Index([10001.0, 10025.0, 10026.0, 10028.0, 10032.0, 10044.0, 10051.0, 10065.0,
       10104.0, 10107.0,
       ...
       93420.0, 93422.0, 93423.0, 93426.0, 93427.0, 93428.0, 93429.0, 93433.0,
       93434.0, 93436.0],
      dtype='float64', name='permno', length=5767)
Index([10001.0, 10025.0, 10026.0, 10028.0, 10032.0, 10044.0, 10065.0, 10104.0,
       10107.0, 10116.0,
       ...
       93420.0, 93422.0, 93423.0, 93426.0, 93427.0, 93428.0, 93429.0, 93433.0,
       93434.0, 93436.0],
      dtype='float64', name='permno', length=5749)
Index([10001.0, 10025.0, 10026.0, 10028.0, 10032.0, 10044.0, 10065.0, 10104.0,
       10107.0, 10116.0,
       ...
       93420.0, 93422.0, 93423.0, 93426.0, 93427.0, 93428.0, 93429.0, 93433.0,


CA3 Inferencing & Predicting: 100%|██████████| 30/30 [25:44<00:00, 51.49s/it, Year=2016]


ValueError: cannot reindex on an axis with duplicate labels